In [76]:
import csv
import pandas as pd
import numpy as np
from sklearn import preprocessing
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import Normalizer

In [77]:
# baca data csv , simpan ke bentuk list
with open("Fitur/area.csv") as f:
    reader = csv.reader(f)
    # fungsi list() untuk mengubah ke bentuk list()
    area = list(reader)[0]
    
with open("Fitur/eccentricity.csv") as f:
    reader = csv.reader(f)
    eccentricity = list(reader)[0]

with open("Fitur/perimeter.csv") as f:
    reader = csv.reader(f)
    perimeter = list(reader)[0]
    
with open("Fitur/metric.csv") as f:
    reader = csv.reader(f)
    metric = list(reader)[0]
    
with open("Fitur/green.csv") as f:
    reader = csv.reader(f)
    green = list(reader)[0]
    
with open("Fitur/label.csv") as f:
    reader = csv.reader(f)
    label = list(reader)[0]
    
# ganti label dalam bentuk string
idx = 0
for i in label:
    if i == "1":
        label[idx] = "Cocor Bebek"
    elif i == "2":
        label[idx] = "Jambu"
    elif i == "3":
        label[idx] = "Kacang Panjang"
    elif i == "4":
        label[idx] = "Singkong"
    elif i == "5":
        label[idx] = "Tomat"
        
    idx = idx+1

In [79]:
import pandas as pd
# fungsi map() untuk mengubah tipe data semua element dalam list
# fungsi zip() untuk menggandengkan setiap list menjadi matriks
df = pd.DataFrame(list(zip(list(map(int,area)), list(map(float,eccentricity)), list(map(float,perimeter)), list(map(float,metric)), list(map(float,green)), label)), columns = ['Area', 'Eccentricity', 'Perimeter', 'Metric', 'Green', 'Label'])

In [87]:
# sekedar melihat struktur tabel
df.sample()

,Area,Eccentricity,Perimeter,Metric,Green,Label
4,3802,0.823976,447.988,0.238062,52.947575,Cocor Bebek


In [81]:
# buat encoder
le = preprocessing.LabelEncoder()

In [82]:
# memisahkan data ke train dan test
train_dataset = df.sample(frac=0.8, random_state=0)
test_dataset = df.drop(train_dataset.index)

In [83]:
# pisahkan feature dari label
area_train = train_dataset['Area'].tolist()
eccentricity_train = train_dataset['Eccentricity'].tolist()
perimeter_train = train_dataset['Perimeter'].tolist()
metric_train = train_dataset['Metric'].tolist()
green_train = train_dataset['Green'].tolist()

# membuat nilai numerik untuk setiap label
label_train = le.fit_transform(train_dataset['Label'])

area_test = test_dataset['Area'].tolist()
eccentricity_test = test_dataset['Eccentricity'].tolist()
perimeter_test = test_dataset['Perimeter'].tolist()
metric_test = test_dataset['Metric'].tolist()
green_test = test_dataset['Green'].tolist()

label_test = le.fit_transform(test_dataset['Label'])

In [84]:
# menyiapkan fitur data train dan testing, (menggabungkan tiap variabel fitur)
train_features = list(zip(area_train, eccentricity_train, perimeter_train, metric_train, green_train))

test_features = list(zip(area_test, eccentricity_test, perimeter_test, metric_test, green_test))

In [85]:
# buat model KNN dengan jumlah parameter tetangga = 7
model = KNeighborsClassifier(n_neighbors=7)

model.fit(train_features, label_train)

acc = model.score(test_features, label_test)
print("accuracy : ", acc*100)

accuracy :  70.0


In [88]:
# percobaan prediksi
X_predict = [3802,	0.823976,	447.988,	0.238062,	52.947575]

transformer = Normalizer().fit([X_predict])
X_normalize = transformer.transform([X_predict])

predict_class = model.predict([X_predict])

if predict_class[0] == 0:
    print("prediction class : Cocor Bebek")
elif predict_class[0] == 1:
    print("prediction class : Jambu")
elif predict_class[0] == 2:
    print("prediction class : Kacang Panjang")
elif predict_class[0] == 3:
    print("prediction class : Singkong")
elif predict_class[0] == 4:
    print("prediction class : Tomat")

prediction class : Cocor Bebek
